# Create Word List 1000

元の文章テキストファイルから単語を 1,000 個抽出する。
用途に合わせて2種類の形態素解析器を使用することができる。

In [ ]:
import os

def read_all_txt_files(directory):
    combined_text = ""
    # ディレクトリ内の全てのファイルを走査
    for root, _, files in os.walk(directory):
        for file in files:
            # 拡張子が.mdのファイルのみ対象
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                with open(file_path, "r", encoding="utf-8") as f:
                    combined_text += f.read() + "\n"  # ファイル内容を結合
    return combined_text

# 使用例
directory_path = "./data"  # ディレクトリのパスを指定
all_text = read_all_txt_files(directory_path)

len(all_text)

In [ ]:
import json
# JSONファイルからストップワードを読み込む
with open("./custom_stopwords_ja.json", encoding="utf-8") as f:
    stopwords = set(json.load(f)["stopwords"])


# Sudachi Extractor
https://github.com/WorksApplications/SudachiPy

In [ ]:
import sudachipy
from sudachipy import dictionary
import pandas as pd
from collections import Counter
import re
# 形態素解析器のインスタンス作成
tokenizer = dictionary.Dictionary().create()

# 名詞カウンター
noun_counter = Counter()

# 名詞（普通名詞）の抽出
for line in all_text.split('\n'):
    if line.strip():
        for morpheme in tokenizer.tokenize(line, mode=tokenizer.SplitMode.C):
            pos = morpheme.part_of_speech()

            # 「名詞」「普通名詞」のチェック
            if pos[0] == "名詞" and pos[1] == "普通名詞":
                word = morpheme.surface()
                if not re.fullmatch(r'^[a-zA-Z0-9]+$', word) and word not in stopwords:
                    noun_counter[word] += 1

            # 1000個集まったら終了
            if len(noun_counter) >= 1000:
                break

# 上位1000個の名詞を取得
top_nouns = noun_counter.most_common(1000)

# データフレームに変換
df = pd.DataFrame(top_nouns, columns=['Noun', 'Frequency'])

# CSVとして保存
df.to_csv('./embeddings/busho_nouns_sudachi_1000.csv', index=False, encoding='utf-8')



# MeCab + Neologd

https://github.com/neologd/mecab-ipadic-neologd/blob/master/README.ja.md

In [ ]:
import json
import MeCab
import pandas as pd
from collections import Counter

# MeCabの形態素解析器を生成（NEologd辞書を指定）
mecab = MeCab.Tagger("-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd")

# 名詞カウンター
noun_counter = Counter()

# サンプルテキスト
# sample_text = """
# 私たちはAI技術を使って新しいソリューションを開発しています。
# 東京都内の企業が集まるイベントでプレゼンテーションを行いました。
# スポーツ、文化、教育、科学、技術など、さまざまな分野における研究が進んでいます。
# 社会問題解決のために、地域社会との連携が重要です。
# 最新技術を用いた製品開発が進行中です。PythonやJavaScriptの勉強も必要です。
# """

# 名詞抽出とストップワード除去
for line in all_text.split('\n'):
    if line.strip():
        parsed = mecab.parse(line)
        for row in parsed.split('\n'):
            if row == 'EOS' or row == '':
                continue
            cols = row.split('\t')
            if len(cols) > 1:
                word = cols[0]
                pos = cols[1].split(',')

                # 名詞かつストップワードでないものを抽出
                # and pos[1] == "固有名詞" でもいい
                if pos[0] == "名詞"  and word not in stopwords:
                    # 英字や数字、%を含まないものを除外
                    if not re.fullmatch(r'^[a-zA-Z0-9%]+$', word) and not re.fullmatch(r'.*\d+.*', word) and not re.fullmatch(r'^[a-zA-Z\W]+$', word):
                        noun_counter[word] += 1

            # 1000個集まったら終了
            if len(noun_counter) >= 5000:
                break

# 上位1000個の名詞を取得
top_nouns = noun_counter.most_common(1000)

# データフレームに変換
df = pd.DataFrame(top_nouns, columns=['Noun', 'Frequency'])

# CSVとして保存
df.to_csv('./embeddings/busho_nouns_neologd_1000.csv', index=False, encoding='utf-8')
